In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import socket

import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *#avg, count, expr
from pyspark.sql.types import *

In [2]:
# initialize
sc = pyspark.SparkContext()
spark = SparkSession(sc)
spark.sparkContext.appName = 'streaming'
# show the number of cores
print('%d cores'%spark._jsc.sc().getExecutorMemoryStatus().keySet().size())
spark

1 cores


In [3]:
# setup to read the tweets from the socket
listenHost = '127.0.0.1' 
listenPort = 4208
tweets = spark.readStream.format('socket').options(host=listenHost, port=listenPort).load()\
    .selectExpr("CAST(value AS STRING)")

In [8]:
''' get the tweets - user and tweet '''
sep = '¬'
# data prep
tweetsTable = tweets.withColumn('user', split(col('value'), sep).getItem(0))\
    .withColumn('date', split(col('value'), sep).getItem(1))\
    .withColumn('tweet', split(col('value'), sep).getItem(2))#\
    #.drop('value')
# start the streaming
writeTweet = tweetsTable.writeStream.outputMode('append').format('memory').queryName('tweetquery').trigger(processingTime='2 seconds').start()
print('Tweets streaming')

Tweets streaming


In [ ]:
''' get tweets - count the words '''
# data prep
tweetsTable = tweets.withColumn('word', explode(split(col('value'), ' '))).filter(col('word').contains('#'))\
    .groupBy('word').count().sort('count', ascending=False)
# start the streaming
writeTweet = tweetsTable.writeStream.outputMode('complete').format('memory').queryName('tweetquery').trigger(processingTime='2 seconds').start()
print('Tweets streaming')

In [10]:
spark.sql('select count(*) from tweetquery').show()
spark.sql('select user, date, tweet from tweetquery').show(n=100, truncate=False)

+--------+
|count(1)|
+--------+
|      28|
+--------+

+------------------------------------------------------------------------------------------------------------+------------------------------+--------------------------------------------------------------------------------------------------------------+
|user                                                                                                        |date                          |tweet                                                                                                         |
+------------------------------------------------------------------------------------------------------------+------------------------------+--------------------------------------------------------------------------------------------------------------+
|dapimpisere15                                                                                               |Sun Aug 01 20:30:23 +0000 2021|RT @1DAFSanta: this is what the crowd at a o

In [11]:
writeTweet.stop()

In [12]:
sc.stop()